In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load the dataset
df = pd.read_csv('bootcamp_student_data_filled.csv')

# Fill missing values
df = df.fillna(0)

# Define target and features
y = df['Bootcamp_Interest']
X = df.drop(['Name', 'Bootcamp_Interest', 'DOB', 'Time_to_Graduate'], axis=1)

In [34]:
# Feature Scaling
standard_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

X[['Years_of_Work_Experience', 'LinkedIn_Connections']] = minmax_scaler.fit_transform(
    X[['Years_of_Work_Experience', 'LinkedIn_Connections']]
)
X[['Certifications']] = standard_scaler.fit_transform(X[['Certifications']])

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df['Standardized_Technical_Skills'] = df['Relevant_Technical_Skills'].apply(lambda x: 1 if x >= 5 else 0)
df.drop('Relevant_Technical_Skills', axis=1, inplace=True)


In [35]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=10, validation_split=0.1, verbose=1)

# Evaluate the model
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Generate classification report
report = classification_report(y_test, y_pred)
print(report)


c:\Users\me\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - accuracy: 0.1619 - loss: 0.7361 - val_accuracy: 0.0000e+00 - val_loss: 0.7518
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2905 - loss: 0.6947 - val_accuracy: 1.0000 - val_loss: 0.6789
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7571 - loss: 0.6525 - val_accuracy: 1.0000 - val_loss: 0.6225
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7571 - loss: 0.6302 - val_accuracy: 1.0000 - val_loss: 0.5727
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7905 - loss: 0.5985 - val_accuracy: 1.0000 - val_loss: 0.5325
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7905 - loss: 0.5853 - val_accuracy: 1.0000 - val_loss: 0.5028
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8238 - loss: 0.5392 - val_accuracy: 1.0000 - val_loss: 0.4815
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7571 - loss: 0.5853 - val_accuracy: 1.0000 - val_loss: 0.

In [37]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [38]:
# Add predictions to DataFrame
df['Predicted_Graduates'] = 0
df.loc[X_test.index, 'Predicted_Graduates'] = y_pred

# Filter rows where 'Predicted_Graduates' is 1
filtered_df = df[df['Predicted_Graduates'] == 1][['Name', 'Predicted_Graduates']]
print(filtered_df.reset_index(drop=True))



           Name  Predicted_Graduates
0  Ragland, Joy                    1


In [39]:
# Save the updated DataFrame
df.to_csv('bootcamp_grad_nn_mod_predictions.csv', index=False)
